## Appendix A.MIST Grid

The MIST grid is a precomputed library of stellar evolution models, tabulated over mass, age, and metallicity.
Each grid point represents a physically consistent stellar state predicted by the MESA code.
In this practicum, stellar ages are inferred by matching observed parameters to this grid, treating it as a lookup table rather than a fitting algorithm.

In [ ]:
# =========================================================
# Cell 0 — Notebook bootstrap (globals, paths, theme, i18n)
# =========================================================

from __future__ import annotations

from pathlib import Path
from functools import partial
import importlib

import matplotlib.pyplot as plt

from lulab.io.paths import (
    get_topic_root,
    figures_dir,
    data_raw_dir,
    data_processed_dir,
    animations_dir,
)
from lulab.io.save_figure import save_fig
import lulab.io.theme as th

# -----------------------------
# Global notebook constants
# -----------------------------
TOPIC: str = "TOP_0001_exoplanet_birth_radius"
NOTEBOOK: str = "ACAP_001"
LANG: str = "en"   # "en" / "ru"

# --- physical limits (keep consistent) ---
AGE_MIN_GYR = 0.1
AGE_MAX_GYR = 13.5

# --- constants ---
R_SUN = 8.0  # kpc

# -----------------------------
# Theme
# -----------------------------
th.set_theme("light")   # или th.apply_theme(th.THEME)

# -----------------------------
# Common paths
# -----------------------------
TOPIC_ROOT: Path = get_topic_root(TOPIC)
FIG_DIR: Path = figures_dir(TOPIC, lang=LANG, create=True)
DATA_RAW_DIR: Path = data_raw_dir(TOPIC, create=False)         # raw обычно уже существует
DATA_PROCESSED_DIR: Path = data_processed_dir(TOPIC, create=True)
ANIM_DIR: Path = animations_dir(TOPIC, create=True)

# -----------------------------
# Save-figure helper bound to this topic/lang
# -----------------------------
# save_fig0 = partial(save_fig, topic=TOPIC, lang=LANG)

# -----------------------------
# Save-figure helper bound to this topic/lang/notebook
# -----------------------------
def save_fig_nb(fig_id: str, *, fig):
    """
    Save figure with notebook-scoped ID to avoid collisions.

    Example:
        save_fig0("Figure_3", fig=fig)
    -> saved as:
        ACAP_001_Figure_3.png
    """
    full_id = f"{NOTEBOOK}_{fig_id}"
    return save_fig(full_id, fig=fig, topic=TOPIC, lang=LANG)

save_fig0 = save_fig_nb

# -----------------------------
# i18n bootstrap (force reload to avoid stale imports)
# -----------------------------
import lulab.i18n.plot_text as pt
importlib.reload(pt)

print("plot_text module path:", pt.__file__)
print("has set_notebook:", hasattr(pt, "set_notebook"))

pt.set_lang(LANG)
if hasattr(pt, "set_notebook"):
    pt.set_notebook(NOTEBOOK)

pt.load_topic_i18n(TOPIC_ROOT, strict=True)
pt.debug_i18n_state()

# Convenience aliases for notebook use:
L = pt.L
T = pt.T

# -----------------------------
# Quick sanity printout
# -----------------------------
print("\n--- BOOTSTRAP ---")
print("THEME     :", th.THEME)
print("TOPIC     :", TOPIC)
print("NOTEBOOK  :", NOTEBOOK)
print("LANG      :", LANG)
print("TOPIC_ROOT:", TOPIC_ROOT)
print("FIG_DIR   :", FIG_DIR)
print("DATA_RAW  :", DATA_RAW_DIR)
print("DATA_PROC :", DATA_PROCESSED_DIR)
print("ANIM_DIR  :", ANIM_DIR)

plot_text module path: /Users/mloktionov/PycharmProjects/Stellar_Attractor/attractor-lab/core/src/lulab/i18n/plot_text.py
has set_notebook: True
lang: en
notebook: ACAP_001
loaded_from: /Users/mloktionov/PycharmProjects/Stellar_Attractor/attractor-lab/topics/TOP_0001_exoplanet_birth_radius/i18n
labels(top-level keys): 2
titles(top-level keys): 2

--- BOOTSTRAP ---
THEME     : light
TOPIC     : TOP_0001_exoplanet_birth_radius
NOTEBOOK  : ACAP_001
LANG      : en
TOPIC_ROOT: /Users/mloktionov/PycharmProjects/Stellar_Attractor/attractor-lab/topics/TOP_0001_exoplanet_birth_radius
FIG_DIR   : /Users/mloktionov/PycharmProjects/Stellar_Attractor/attractor-lab/topics/TOP_0001_exoplanet_birth_radius/figures/en
DATA_RAW  : /Users/mloktionov/PycharmProjects/Stellar_Attractor/attractor-lab/topics/TOP_0001_exoplanet_birth_radius/data/raw
DATA_PROC : /Users/mloktionov/PycharmProjects/Stellar_Attractor/attractor-lab/topics/TOP_0001_exoplanet_birth_radius/data/processed
ANIM_DIR  : /Users/mloktionov/Py

### Quick peek into the MIST grid (what is inside `mist_grid_cache.parquet`?)

This cell is purely exploratory: we load the cached MIST grid and inspect its schema and basic parameter ranges to understand what the “grid” physically is (a large table of precomputed stellar-evolution model points).


In [ ]:
# =========================================================
# [ACAP_003|MIST] Inspect cached MIST grid (schema + head)
# =========================================================

import numpy as np
import pandas as pd

grid_path = DATA_PROCESSED_DIR / "mist_grid_cache.parquet"
grid = pd.read_parquet(grid_path)

print("MIST grid file:", grid_path)
print("Shape:", grid.shape)
print("\nColumns (N={}):".format(len(grid.columns)))
print(list(grid.columns))

print("\n--- HEAD ---")
display(grid.head(10))

# Some common columns we often rely on in this practicum
maybe_cols = ["age", "age_gyr", "feh", "Teff", "logg", "mass", "logL", "radius", "phase"]
present = [c for c in maybe_cols if c in grid.columns]
print("\nCommon columns present:", present)

# If age is stored as log10(years), create a helper "age_gyr" for inspection
if "age" in grid.columns and "age_gyr" not in grid.columns:
    grid = grid.copy()
    grid["age_gyr"] = (10 ** grid["age"]) / 1e9

# Show basic ranges for the columns we actually use in the notebook
stat_cols = [c for c in ["age_gyr", "feh", "Teff", "logg"] if c in grid.columns]
print("\n--- BASIC RANGES (used in our fits) ---")
for c in stat_cols:
    s = pd.to_numeric(grid[c], errors="coerce")
    print(
        f"{c:8s}: "
        f"min={np.nanmin(s):.4g}, "
        f"p01={np.nanpercentile(s, 1):.4g}, "
        f"median={np.nanmedian(s):.4g}, "
        f"p99={np.nanpercentile(s, 99):.4g}, "
        f"max={np.nanmax(s):.4g}"
    )

# Optional: check how dense the grid is in metallicity (helps explain why we slice by feh)
if "feh" in grid.columns:
    feh_vals = pd.to_numeric(grid["feh"], errors="coerce").dropna().to_numpy()
    uniq = np.unique(np.round(feh_vals, 4))
    print("\n--- METALLICITY GRID ---")
    print("Unique [Fe/H] values (rounded to 1e-4):", len(uniq))
    print("First 15:", uniq[:15])
    print("Last  15:", uniq[-15:])

# Optional: rough memory footprint
mem_mb = grid.memory_usage(deep=True).sum() / 1024**2
print(f"\nApprox DataFrame RAM usage: {mem_mb:.1f} MB")

MIST grid file: /Users/mloktionov/PycharmProjects/Stellar_Attractor/attractor-lab/topics/TOP_0001_exoplanet_birth_radius/data/processed/mist_grid_cache.parquet
Shape: (1494453, 16)

Columns (N=16):
['eep', 'age', 'feh', 'mass', 'initial_mass', 'radius', 'density', 'logTeff', 'Teff', 'logg', 'logL', 'Mbol', 'delta_nu', 'nu_max', 'phase', 'dm_deep']

--- HEAD ---


,eep,age,feh,mass,initial_mass,radius,density,logTeff,Teff,logg,logL,Mbol,delta_nu,nu_max,phase,dm_deep
0,35,5.0,-3.978406,0.100000,0.100000,1.106082,0.104181,3.617011,4140.105252,3.350571,-0.489734,5.964335,37.987066,299.346079,-1.0,0.002885
1,36,5.0,-3.978406,0.102885,0.102885,1.122675,0.102504,3.618039,4149.909661,3.347798,-0.472691,5.921728,37.739176,298.570836,-1.0,0.003573
2,37,5.0,-3.978406,0.107147,0.107147,1.147702,0.099918,3.619556,4164.436984,3.343658,-0.447471,5.858678,37.345115,297.180748,-1.0,0.004247
3,38,5.0,-3.978406,0.111379,0.111379,1.173015,0.097284,3.621062,4178.903372,3.339612,-0.422498,5.796244,36.923615,295.526946,-1.0,0.004217
4,39,5.0,-3.978406,0.115581,0.115581,1.198615,0.094623,3.622555,4193.289262,3.335660,-0.397776,5.734440,36.473151,293.589960,-1.0,0.004189
5,40,5.0,-3.978406,0.119757,0.119757,1.224529,0.091949,3.624036,4207.619059,3.331790,-0.373270,5.673176,35.991685,291.348026,-1.0,0.004162
6,41,5.0,-3.978406,0.123906,0.123906,1.250725,0.089281,3.625505,4221.874996,3.328014,-0.349009,5.612523,35.478708,288.789642,-1.0,0.004133
7,42,5.0,-3.978406,0.128023,0.128023,1.277139,0.086642,3.626961,4236.048889,3.324363,-0.325034,5.552585,34.935353,285.920858,-1.0,0.004102
8,43,5.0,-3.978406,0.132110,0.132110,1.303717,0.084050,3.628404,4250.149048,3.320864,-0.301371,5.493428,34.362950,282.751756,-1.0,0.004071
9,44,5.0,-3.978406,0.136165,0.136165,1.330388,0.081523,3.629831,4264.135010,3.317555,-0.278074,5.435185,33.764501,279.313278,-1.0,0.004040



Common columns present: ['age', 'feh', 'Teff', 'logg', 'mass', 'logL', 'radius', 'phase']

--- BASIC RANGES (used in our fits) ---
age_gyr : min=0.0001, p01=0.0001413, median=0.3548, p99=19.95, max=19.95
feh     : min=-4.939, p01=-3.979, median=-0.8873, p99=18.86, max=inf
Teff    : min=2211, p01=2724, median=5322, p99=2.096e+05, max=4.012e+05
logg    : min=-1.134, p01=-0.6983, median=2.855, p99=8.17, max=8.64

--- METALLICITY GRID ---
Unique [Fe/H] values (rounded to 1e-4): 71637
First 15: [-4.9389 -4.9322 -4.9175 -4.9151 -4.9093 -4.9028 -4.8882 -4.8813 -4.8687
 -4.8683 -4.8658 -4.8636 -4.8606 -4.8589 -4.8537]
Last  15: [32.9559 32.9576 32.9619 32.986  33.0142 33.0333 33.0632 33.0723 33.0766
 33.0767 33.2853 33.2929 33.303  33.367      inf]

Approx DataFrame RAM usage: 193.8 MB


--- 